In [6]:
%load_ext sql
%sql mysql://root@localhost/rental_nerd

# imports
import pandas as pd
import numpy as np
import datetime as dt
import timeit  # for timing models
import contextlib
from sklearn import metrics
from sklearn import cross_validation as cv
import matplotlib.pyplot as plt

import os
import gc

from slacker import Slacker
import json
import requests
from cloudinary.uploader import upload
from cloudinary.utils import cloudinary_url
from cloudinary.api import delete_resources_by_tag, resources_by_tag

# this allows plots to appear directly in the notebook
%matplotlib inline

import xgboost as xgb

# today's date for output filenames
today = dt.date.today()

# where to save the xgb models - they get huge so keep them out of any git path
model_path = '/home/ilya/rentalnerd-models/'
csv_path = '/home/ilya/Code/rentalnerd/scraper/'

# booster parameters
param = {'verbose': 0,
         'silent': 0,
         'objective':'reg:linear',
         'booster': 'gbtree',
         'eval_metric':'mae', 
         'updater': 'grow_gpu',
         'eta': 0.1, # not tuned, learning rate with default of 0.3
         'max_depth': 10,  # all of the following parameters are __tuned__ so do not change them
         'alpha': 2.6456,
         'gamma': 6.4589, 
         'subsample': 0.9893,
         'colsample_bytree': 0.6759,
         'min_child_weight': 16,
         'max_delta_step': 0
        }

num_round = 5000 # pick a high number - XGB will abort as soon as accuracy drops in the testing set

import os
# slack secrets (in your ~/.bashrc)
webhook_url = os.environ.get('SLACK_URL')
slacker = Slacker(os.environ.get('SLACK_TOKEN'))

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [7]:
def plot_rounds(plot):
    # uploads the graph to the web and returns the URL
    
    fig = plot.get_figure()
    fig.tight_layout()
    fig.savefig('temp_plot.png')
    
    response = upload("temp_plot.png")
    url, options = cloudinary_url(response['public_id'],
        format = response['format'],
        crop = "fill")
    return url

def slack(text, url = None, title = None):
    print("Slacking: " + text)
    
    if url == None:
        data=json.dumps({"text": text})
    else:
        data = json.dumps( { "text": text, "attachments": [ { "fallback": "Model MAE"
                                           , "title": title
                                           , "image_url": url } ] } )
    
    response = requests.post(webhook_url, data , headers={'Content-Type': 'application/json'})
    if response.status_code != 200:
        raise ValueError('Request to slack returned an error %s, the response is:\n%s' % (response.status_code, response.text))

        
def output_model_metrics( x, ypred, y_known, t ):
    #Print model report:
    mae = metrics.mean_absolute_error(y_known, ypred)
    r2 = metrics.explained_variance_score(y_known, ypred)
  
    slack("Model Report:\t%s \t n:\t%i \t\t MAE Score:\t%f \t\t R^2:\t%f" % (t, len(y_known), mae, r2))

    
def train_model(train, test, factors, label, xgb_model = None):
    dtrain = xgb.DMatrix(train[factors].values, label=train[label], feature_names=factors)
    dtest = xgb.DMatrix(test[factors].values, label=test[label], feature_names=factors)
    watchlist  = [(dtrain,'train'),(dtest,'eval')]
    progress = dict()
  
    xgb_model = xgb.train( param, dtrain, num_round, evals = watchlist, xgb_model = xgb_model, evals_result=progress
                        , early_stopping_rounds = 10, verbose_eval = 20 )
        
    if hasattr(xgb_model, 'best_score'):
        slack("Early stopping occured, best_score %f, best_iteration %i" % (xgb_model.best_score, xgb_model.best_iteration))

    curve = pd.DataFrame()
    curve['test'] = progress['eval']['mae']
    curve['train'] = progress['train']['mae']

    url = plot_rounds(curve.plot())
    slack("", url, "MAE by Round ($)")
    
    url = plot_rounds(xgb.plot_importance(xgb_model,max_num_features=20))
    slack("", url, "Feature Importance (n trees)")
        
    # predict the training set using the model - note this is in sample testing
    ypred = xgb_model.predict(dtrain, ntree_limit=xgb_model.best_ntree_limit)
    output_model_metrics( dtrain, ypred, train[label], 'train' )

    # predict the testing set using the model
    ypred = xgb_model.predict(dtest, ntree_limit=xgb_model.best_ntree_limit)
    output_model_metrics( dtest, ypred, test[label], 'test' )
    
    # clean out the model from memory
    xgb_model.save_model(model_path +  'all_' + label + '_' + today.strftime('%Y%m%d') + '.model')
    del xgb_model
    gc.collect()    

def queue_reads():
    # read in all of the files in the same order we ran queries
    sales = pd.read_csv('CSV_backups/ALL-sales.csv',nrows=limit, index_col=0)
    for_sale = pd.read_csv('CSV_backups/ALL-for_sale.csv',nrows=limit, index_col=0)
    
    sales_train, sales_test = cv.train_test_split(sales, test_size = 0.2) # set aside X% of the dataset for testing
    del sales

    return sales_train, sales_test, for_sale

In [8]:
city = 'ALL'
limit = 10000000

sales_train, sales_test, for_sale = queue_reads()
    
limit = min(limit, len(sales_train.index))
    
ind2remove = ['Unnamed: 0', 'address', 'area_name', 'date_listed', 'id', 'listed_diff_id', 'lookup_address',
              'origin_url', 'neighborhood', 'zipcode', 'luxurious', 'transaction_status', 'transaction_type',
              'images', 'date_transacted_latest']
factors = np.setdiff1d(sales_train.columns, ind2remove).tolist()


FileNotFoundError: File b'CSV_backups/ALL-sales.csv' does not exist

In [9]:
# cap number of homes that fit into VRAM
memory_cap = 250000

# init empty model that we can load into on the second iteration
bst = xgb.Booster()

# first run the price model
label = 'price'
f = factors
f.remove(label) # this happens in place

start_time = timeit.default_timer()

for g, df in sales_train.groupby(np.arange(len(sales_train.index)) // memory_cap):  # split the dataset into 250k chunks    
    train_model(df, sales_test, f, label, xgb_model = (bst if g > 0 else None))

    # load the model into memory - should have been saved by train_model function
    bst.load_model(model_path +  'all_' + label + '_' + today.strftime('%Y%m%d') + '.model')

elapsed = timeit.default_timer() - start_time
slack("%s:\tTime to train:\t%f minutes" % (city, (elapsed / 60)))

target = xgb.DMatrix( for_sale[f].values, feature_names=f)
ypred = bst.predict(target, ntree_limit=(bst.best_iteration if hasattr(bst, 'best_score') else None))

# second run the days on the market model
sales_train = sales_train[(sales_train.days_on_market > 0 )]
sales_test = sales_test[(sales_test.days_on_market > 0 )]
label = 'days_on_market'
f = factors
f.remove(label)

for g, df in sales_train.groupby(np.arange(len(sales_train)) // memory_cap):  # split the dataset into 100k chunks    
    train_model(df, sales_test, f, label, xgb_model = (bst if g > 0 else None))

    # load the model into memory - should have been saved by train_model function
    bst.load_model(model_path +  'all_' + label + '_' + today.strftime('%Y%m%d') + '.model')

target = xgb.DMatrix( for_sale[f].values, feature_names=f)
dayspred = bst.predict(target, ntree_limit=(bst.best_iteration if hasattr(bst, 'best_score') else None))

NameError: name 'factors' is not defined

In [10]:
values = np.column_stack((for_sale.index.values
                         ,for_sale.transaction_id.values
                         ,for_sale.address.values
                         ,ypred-for_sale.price
                         ,ypred
                         ,for_sale.price.values
                         ,for_sale['origin_url'].values))
output = pd.DataFrame(values[:,1:],index=values[:,0],columns=['transaction_id', 'address','ypred',
                                                              'predicted_price', 'list', 'url'])
output.index.name = 'property_id'
output = output.sort_values(by='ypred',ascending=False)


# save target list
file = csv_path + 'value_buy/target_list_' + today.strftime('%Y%m%d') + '.csv'
output.to_csv(file)
slacker.files.upload(file, channels='#progress')

NameError: name 'for_sale' is not defined

In [11]:
output.head()

NameError: name 'output' is not defined